In [2]:
pip install torch==2.4.0 torchvision==0.19.0+cu121 torchaudio==2.4.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.nn.utils.rnn import pad_sequence
from nltk.tokenize import word_tokenize
import nltk

# Загрузка ресурса для токенизации
nltk.download('punkt')

# Шаг 1: Загрузка и очистка данных
df = pd.read_csv('/content/train.csv')

# Очистка данных
df['tweet'] = df['tweet'].str.replace(r'@[\w]*', '', regex=True)
df['tweet'] = df['tweet'].str.replace(r'http\S+', '', regex=True)
df['tweet'] = df['tweet'].str.replace(r'[^A-Za-z\s]', '', regex=True)

# Шаг 2: Токенизация текста и подготовка данных
tweets = df['tweet'].apply(word_tokenize).values
labels = df['label'].values

# Токенизация вручную
word2idx = {}
idx2word = {}
vocab_size = 5000
idx = 2  # Начинаем с 2, т.к. 0 — для паддинга, 1 — для неизвестных слов

for tweet in tweets:
    for word in tweet:
        if word not in word2idx and len(word2idx) < vocab_size - 2:
            word2idx[word] = idx
            idx2word[idx] = word
            idx += 1

# Преобразование токенов в индексы
def encode_tweet(tweet, word2idx):
    return [word2idx.get(word, 1) for word in tweet]  # 1 — индекс для неизвестных слов

encoded_tweets = [torch.tensor(encode_tweet(tweet, word2idx), dtype=torch.long) for tweet in tweets]
labels = torch.tensor(labels, dtype=torch.float32)

# Шаг 3: Паддинг последовательностей
padded_tweets = pad_sequence(encoded_tweets, batch_first=True, padding_value=0)

# Шаг 4: Разделение данных на обучающие и тестовые
X_train, X_test, y_train, y_test = train_test_split(padded_tweets, labels, test_size=0.2, random_state=42)

# Шаг 5: Создание пользовательского Dataset и DataLoader
class TweetDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TweetDataset(X_train, y_train)
test_dataset = TweetDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Шаг 6: Определение модели LSTM
class SentimentLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        super(SentimentLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, batch_first=True, dropout=dropout, bidirectional=False)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, (hidden, cell) = self.lstm(embedded)
        hidden = self.dropout(hidden[-1])
        output = self.fc(hidden)
        return torch.sigmoid(output)

# Шаг 7: Инициализация модели
embedding_dim = 100
hidden_dim = 128
output_dim = 1
n_layers = 2
dropout = 0.3

model = SentimentLSTM(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout)

# Шаг 8: Определение оптимизатора и функции потерь
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Шаг 9: Обучение модели
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        predictions = model(X_batch).squeeze()
        loss = criterion(predictions, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

# Шаг 10: Оценка модели
def evaluate(model, test_loader, device):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            predictions = model(X_batch).squeeze()
            y_pred.extend(predictions.cpu().numpy())
            y_true.extend(y_batch.numpy())

    y_pred = np.array([1 if pred > 0.5 else 0 for pred in y_pred])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

# Шаг 11: Обучение модели на нескольких эпохах
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

n_epochs = 5
for epoch in range(n_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    test_accuracy = evaluate(model, test_loader, device)
    print(f'Epoch {epoch+1}/{n_epochs} - Training Loss: {train_loss:.4f} - Test Accuracy: {test_accuracy * 100:.2f}%')

# Шаг 12: Функция для предсказания сентимента для твитов без меток
def predict_sentiment(model, tweets, word2idx, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for tweet in tweets:
            encoded_tweet = torch.tensor([word2idx.get(word, 1) for word in word_tokenize(tweet)], dtype=torch.long)
            # Ensure the tensor is 2D: (1, sequence_length)
            encoded_tweet = encoded_tweet.unsqueeze(0).to(device)

            # Forward pass
            prediction = model(encoded_tweet).item()
            sentiment = 'позитивный' if prediction > 0.5 else 'негативный'
            predictions.append(f"Твит: {tweet}, Предсказание сентимента: {sentiment}")

    return predictions


# Пример использования с тестовым датасетом без меток
test_df = pd.read_csv('/content/test.csv')
test_df['tweet'] = test_df['tweet'].str.replace(r'@[\w]*', '', regex=True)
test_df['tweet'] = test_df['tweet'].str.replace(r'http\S+', '', regex=True)
test_df['tweet'] = test_df['tweet'].str.replace(r'[^A-Za-z\s]', '', regex=True)
test_tweets = test_df['tweet'].values

# Вывод предсказаний
predictions = predict_sentiment(model, test_tweets, word2idx, device)
for prediction in predictions:
    print(prediction)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Выходные данные были обрезаны до нескольких последних строк (5000).
Твит:   audusd better bid reattempts  on aus data   blog silver gold forex, Предсказание сентимента: негативный
Твит: well be there at hrss and recex  will you  ms , Предсказание сентимента: негативный
Твит: officially at the airpo boarding about to sta woohoo    travel backpackerli  , Предсказание сентимента: негативный
Твит:  livelypics sometimes the best way to solve a problem is to just stop caring   motivation, Предсказание сентимента: негативный
Твит: yr ben    visiting his manor  maidstone wks     , Предсказание сентимента: негативный
Твит: i hate life and helen and all her stupid antics i hate her so much  , Предсказание сентимента: негативный
Твит:  by occupation some israelis mean west bank amp maybe gaza palestinians mean all of that  the entire state of israel  , Предсказание сентимента: негативный
Твит: a fantastic performance by  s beating wba today development   coaches deservedwin, Предсказание сентимен

##Возможные пути улучшения модели:

1. Балансировка данных :

Если данные несбалансированы (например, больше относится к одному классу), это может привести к тому, что модель будет «переобучаться» на более частом уровне. Проверьте, насколько классы сбалансированы, и, если необходимо, подтвердите методы балансировки, такие как:

* Передискретизация меньшего класса.
* Недостаточная выборка высшего класса.
* Использование взвешенной функции приводит к потерям, например, передав весовые классы в BCEWithLogitsLoss.

2.  Использование предобученных эмбеддингов :

Вместо случайных эмбеддингов можно использовать предобученные, такие как GloVe или Word2Vec.


3. Увеличение сложности модели :

- Двунаправленный LSTM : Двунаправленные LSTM (или ГРУ) могут захватывать контекст как с начала, так и с конца предложения.
- Увеличение количества слоев : если у вас только один два или LSTM-слоя, можно попробовать увеличить их количество.
- Увеличение размера скрытых слоев : увеличение количества нейронов в скрытых слоях может улучшить способность моделировать более сложные зависимости.

4. Regularization :

- Отсев : значения увеличения dropout(например, до 0,3-0,5) могут помочь избежать переобучения.
- L2-регуляризация : Добавление L2-регуляризации параметров модели может помочь моделям стать более обсуждающей.
5. Гиперпараметры :

- Увеличьте количество эпох, чтобы проверить, увеличиться ли модель дальше.
- Попробуйте изменить скорость обучения (можно попробовать уменьшить ее, если модель «застряла»).
- Попробуйте другие оптимизаторы, такие как RMSProp или SGD с моментумом.
6. Увеличение данных :

 если ваш набор данных небольшой, вы можете искусственно увеличить его, используя методы увеличения данных, такие как добавление шума в текст или замена синонимов.